LeNet模型

In [28]:
import torch
from torch import nn,optim

class leNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,6,5),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(6,16,5),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.fc=nn.Sequential(
            nn.Linear(16*4*4,120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )
    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output
net=leNet()
print(net)

leNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


获取数据

In [29]:
import torchvision
from torchvision import transforms
#获取数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transforms.ToTensor())
#读取数据集
batchSize=256
trainIter=torch.utils.data.DataLoader(mnist_train,batch_size=batchSize,shuffle=True,num_workers=8)
testIter=torch.utils.data.DataLoader(mnist_test,batch_size=batchSize,shuffle=True,num_workers=8)

评价

In [30]:
def evaluate_accuracy(data_iter, net):
    device=torch.device('cuda')
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        net.eval() # 评估模式
        for X, y in data_iter:
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().item()
            n += y.shape[0]
        net.train() # 改回训练模式 
    return acc_sum / n

训练

In [31]:
lr,epochsNum = 0.001,10
net=net.cuda()
loss=torch.nn.CrossEntropyLoss()
optimzer=torch.optim.Adam(net.parameters(),lr)

for epoch in range(epochsNum):
    train_l_sum=n=train_acc_sum=0
    for x,y in trainIter:
        x=x.cuda()
        y=y.cuda()
        yP=net(x)
        l=loss(yP,y)
        l.backward()
        optimzer.step()
        optimzer.zero_grad()
        train_l_sum += l.item()
        train_acc_sum += (yP.argmax(dim=1) == y).sum().item()
        n +=y.shape[0]
    print(epoch,train_l_sum/n,train_acc_sum/n,evaluate_accuracy(testIter,net))


0 0.0072719895362854 0.3091333333333333 0.5766
1 0.003682773479819298 0.6391 0.692
2 0.00293666110932827 0.7208333333333333 0.7352
3 0.002610256784160932 0.7451833333333333 0.7499
4 0.002403641369442145 0.7610166666666667 0.7597
5 0.002246731857955456 0.7755666666666666 0.7752
6 0.0021298543353875476 0.7870166666666667 0.786
7 0.0020331645419200263 0.7976166666666666 0.7954
8 0.0019433643167217573 0.8082 0.8036
9 0.0018690082530180613 0.81865 0.8082
